# **skills_db.json content**

In [ ]:
{
  "programming": ["python", "java", "javascript", "c++", "c#", "ruby", "php", "swift", "kotlin", "go", "rust", "typescript"],
  "web_dev": ["html", "css", "react", "angular", "vue", "node.js", "django", "flask", "express", "fastapi", "spring"],
  "databases": ["sql", "mysql", "postgresql", "mongodb", "oracle", "redis", "cassandra", "dynamodb", "sqlite"],
  "ml_ai": ["machine learning", "deep learning", "artificial intelligence", "tensorflow", "pytorch", "keras", "scikit-learn", "sklearn", "nlp", "computer vision", "opencv"],
  "cloud": ["aws", "azure", "gcp", "cloud", "ec2", "s3", "lambda", "kubernetes", "docker", "gke", "aks", "eks"],
  "tools": ["git", "github", "gitlab", "jenkins", "jira", "confluence", "docker", "kubernetes", "terraform", "ansible"],
  "methodologies": ["agile", "scrum", "devops", "ci/cd", "tdd", "waterfall"],
  "data": ["data analysis", "data science", "data visualization", "tableau", "power bi", "excel", "pandas", "numpy", "scipy", "statsmodels"],
  "soft_skills": ["leadership", "communication", "teamwork", "problem solving", "analytical", "critical thinking"]
}

# **CODE**

In [ ]:
!pip install pandas nltk spacy scikit-learn matplotlib seaborn pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 74.4 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
"""
Advanced Resume Screening and Shortlisting System (Upgraded v2)
Features: NLP, NER, TF-IDF, Cosine Similarity, Robust Experience/Name Extraction, Visualization
"""

import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import pdfplumber
import logging
import warnings # To handle the matplotlib warning

# Suppress noisy logs from pdfminer (used by pdfplumber)
logging.getLogger('pdfminer').setLevel(logging.ERROR)

# --- NLTK Data Downloads ---
def download_nltk_data():
    """Download required NLTK data models if not found."""
    nltk_packages = [
        ('tokenizers/punkt', 'punkt'),
        ('tokenizers/punkt_tab', 'punkt_tab'),
        ('corpora/stopwords', 'stopwords'),
        ('corpora/wordnet', 'wordnet'),
        ('corpora/omw-1.4', 'omw-1.4')
    ]
    for path, package_id in nltk_packages:
        try:
            nltk.data.find(path)
        except LookupError:
            print(f"Downloading NLTK package: {package_id}...")
            nltk.download(package_id)

download_nltk_data()

# --- Load spaCy model for NER ---
def load_spacy_model(model_name="en_core_web_sm"):
    """Load or download spaCy model."""
    try:
        nlp = spacy.load(model_name)
    except:
        print(f"Downloading spaCy model: {model_name}...")
        os.system(f"python -m spacy download {model_name}")
        nlp = spacy.load(model_name)
    return nlp

nlp = load_spacy_model()


class AdvancedResumeScreener:
    """
    Advanced Resume Screening System with NER, TF-IDF, Experience, Education, and Visualization
    """

    def __init__(self, skills_db_path='skills_db.json'):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english')) - {'and', 'to', 'of'}
        self.nlp = nlp
        self.matcher = Matcher(self.nlp.vocab)

        # --- NEW: Blacklist for common non-name headers ---
        self.NAME_BLACKLIST = {
            'linkedin', 'github', 'portfolio', 'profile', 'leetcode', 'email', 'phone',
            'address', 'resume', 'cv', 'work', 'contact', 'experience', 'education',
            'skills', 'projects', 'summary'
        }

        # Education degrees
        self.education_keywords = {
            'phd': ['phd', 'ph.d', 'doctorate', 'doctoral'],
            'masters': ['masters', 'master', 'msc', 'm.sc', 'ma', 'm.a', 'mba', 'm.b.a', 'mtech', 'm.tech'],
            'bachelors': ['bachelors', 'bachelor', 'bsc', 'b.sc', 'ba', 'b.a', 'btech', 'b.tech', 'be', 'b.e'],
            'diploma': ['diploma', 'associate']
        }

        # Load skills from external JSON
        self.technical_skills = self.load_skills_from_json(skills_db_path)

        # Initialize TF-IDF Vectorizer
        self.tfidf_vectorizer = TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 2)  # Capture single words and two-word phrases
        )

    def load_skills_from_json(self, json_path):
        """Loads the skills database from an external JSON file."""
        try:
            with open(json_path, 'r') as f:
                print(f"Loading skills database from {json_path}...")
                return json.load(f)
        except FileNotFoundError:
            print(f"Error: {json_path} not found. Using empty skills DB.")
            return {}
        except json.JSONDecodeError:
            print(f"Error: Could not decode {json_path}. Using empty skills DB.")
            return {}

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF resume using pdfplumber for better layout handling"""
        text = ""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    # Extract text, respecting layout
                    page_text = page.extract_text(x_tolerance=2, y_tolerance=2)
                    if page_text:
                        text += page_text + "\n"
                return text
        except Exception as e:
            print(f"Error reading {pdf_path} with pdfplumber: {str(e)}")
            return ""  # Return empty string on failure

    def preprocess_text(self, text):
        """Preprocess text with cleaning, tokenization, and lemmatization"""
        text = text.lower()
        text = re.sub(r'\\x[a-f0-9]{2}', '', text)  # Remove hex characters
        text = re.sub(r'\\[abtnr]', '', text)     # Remove escape sequences
        text = re.sub(r'[^\w\s]', ' ', text)       # Remove punctuation
        text = re.sub(r'\s+', ' ', text)           # Condense whitespace

        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words and len(token) > 1]
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        return ' '.join(tokens)

    # ---
    # --- THIS FUNCTION IS RECTIFIED ---
    # ---
    def extract_contact_info_and_name(self, text, doc):
        """
        Extracts contact info using regex and a robust, multi-step name extraction logic
        """
        info = {
            'name': 'Unknown',
            'email': 'N/A',
            'phone': 'N/A'
        }

        # --- 1. Extract Name (More Robust Multi-Step Logic) ---

        # Step 1: High-Confidence Search
        # Look for a 'PERSON' entity in the first 75 tokens (the top of the resume)
        person_entities = [ent.text for ent in doc[:75].ents if ent.label_ == 'PERSON']
        if person_entities:
            info['name'] = person_entities[0] # Take the first one

        # Step 2: Blacklist Check
        # Check if the found name is likely a header (e.g., "LinkedIn")
        name_parts = set(info['name'].lower().split())
        if info['name'] == 'Unknown' or name_parts.intersection(self.NAME_BLACKLIST):
            # Name is "Unknown" OR it's a blacklisted word, so we reset and try Fallback 1
            info['name'] = 'Unknown'

            # Fallback 1: Try the Proper Noun (PROPN) Matcher
            # This is what we had before, but now it's only a fallback
            pattern1 = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
            pattern2 = [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]

            if "NAME_PATTERN_1" not in self.matcher._patterns:
                self.matcher.add("NAME_PATTERN_1", [pattern1], on_match=None)
            if "NAME_PATTERN_2" not in self.matcher._patterns:
                self.matcher.add("NAME_PATTERN_2", [pattern2], on_match=None)

            matches = self.matcher(doc[:75])
            if matches:
                matches.sort(key=lambda x: x[2] - x[1], reverse=True) # Get longest match
                match_id, start, end = matches[0]
                potential_name = doc[start:end].text

                # Check blacklist *again*
                potential_name_parts = set(potential_name.lower().split())
                if not potential_name_parts.intersection(self.NAME_BLACKLIST):
                    info['name'] = potential_name

        # Fallback 2: Last Resort
        # If name is *still* Unknown, search the *entire document* for the first PERSON entity
        if info['name'] == 'Unknown':
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    name_parts = set(ent.text.lower().split())
                    if not name_parts.intersection(self.NAME_BLACKLIST):
                        info['name'] = ent.text
                        break  # Take the first valid one

        # --- 2. Extract Email ---
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        email_match = re.search(email_pattern, text)
        if email_match:
            info['email'] = email_match.group(0)

        # --- 3. Extract Phone ---
        # More comprehensive phone regex
        phone_pattern = r'(\(?\+?\d{1,3}\)?[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?[\d\s-]{10,15}\b'
        phone_match = re.search(phone_pattern, text)
        if phone_match:
            info['phone'] = phone_match.group(0).strip()

        return info

    def extract_experience(self, text):
        """
        Extract years of experience from resume by parsing date ranges.
        """
        # Regex to find date ranges
        date_range_pattern = re.compile(
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w.]{0,6}\s+\d{4}|'  # Month YYYY
            r'\d{1,2}[\/.-]\d{4}|'  # MM/YYYY
            r'\b\d{4}\b)'  # YYYY
            r'\s*(?:[-–—toTO])\s*'
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w.]{0,6}\s+\d{4}|'  # Month YYYY
            r'\d{1,2}[\/.-]\d{4}|'  # MM/YYYY
            r'\b\d{4}\b|'  # YYYY
            r'Present|Current|Till Date|Now)',  # Present
            re.IGNORECASE
        )

        matches = date_range_pattern.findall(text)

        if not matches:
            # Fallback to simple "X years experience" if no date ranges found
            exp_patterns = r'(\d+)\+?\s*(?:years?|yrs?)\s*(?:of)?\s*experience'
            exp_match = re.search(exp_patterns, text.lower())
            if exp_match:
                return {'total_years': int(exp_match.group(1))}
            return {'total_years': 0}

        month_map = {
            'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
            'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
        }

        def parse_date(date_str):
            date_str = date_str.lower().strip().replace('.', '')

            # Case 1: Present/Current
            if date_str in ['present', 'current', 'till date', 'now']:
                return datetime.now()

            # Case 2: Month YYYY (e.g., "Jan 2020" or "September 2020")
            for month_name, month_num in month_map.items():
                if date_str.startswith(month_name):
                    year_match = re.search(r'(\d{4})', date_str)
                    if year_match:
                        return datetime(int(year_match.group(1)), month_num, 1)

            # Case 3: MM/YYYY (e.g., "01/2020")
            match = re.match(r'(\d{1,2})[\/.-](\d{4})', date_str)
            if match:
                return datetime(int(match.group(2)), int(match.group(1)), 1)

            # Case 4: YYYY (e.g., "2020")
            match = re.match(r'^(\d{4})$', date_str)
            if match:
                # Assume start of year if just YYYY
                return datetime(int(match.group(1)), 1, 1)

            return None # Could not parse

        parsed_ranges = []
        for start_str, end_str in matches:
            start_date = parse_date(start_str)
            end_date = parse_date(end_str)

            if start_date and end_date:
                parsed_ranges.append((start_date, end_date))

        if not parsed_ranges:
            return {'total_years': 0}

        # --- Handle Overlaps ---
        parsed_ranges.sort(key=lambda x: x[0])

        merged_ranges = []
        if parsed_ranges:
            merged_ranges = [parsed_ranges[0]]
            for current_start, current_end in parsed_ranges[1:]:
                last_start, last_end = merged_ranges[-1]

                if current_start <= last_end:
                    new_end = max(last_end, current_end)
                    merged_ranges[-1] = (last_start, new_end)
                else:
                    merged_ranges.append((current_start, current_end))

        # Calculate total duration from merged, non-overlapping ranges
        total_months = 0
        for start_date, end_date in merged_ranges:
            months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
            total_months += (months + 1) # Add 1 for inclusivity

        return {'total_years': round(total_months / 12, 1)}

    def extract_education(self, text, doc):
        """
        Extract education qualifications from resume
        Returns: Dictionary with education details
        """
        education_info = {
            'highest_degree': 'None',
            'degrees': [],
            'institutions': []
        }
        text_lower = text.lower()
        degree_priority = ['phd', 'masters', 'bachelors', 'diploma']

        for degree in degree_priority:
            for keyword in self.education_keywords[degree]:
                if keyword in text_lower:
                    if education_info['highest_degree'] == 'None':
                        education_info['highest_degree'] = degree.title()
                    if degree.title() not in education_info['degrees']:
                        education_info['degrees'].append(degree.title())

        # Extract university/college names using NER
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                org_lower = ent.text.lower()
                if any(word in org_lower for word in ['university', 'college', 'institute', 'school']):
                    if ent.text not in education_info['institutions']:
                        education_info['institutions'].append(ent.text)

        education_info['institutions'] = list(set(education_info['institutions']))
        return education_info

    def extract_skills(self, text, custom_skills=None):
        """
        Extract skills from text with categorization
        Returns: Dictionary with categorized skills
        """
        text_lower = text.lower()
        found_skills = {
            'all_skills': [],
            'by_category': {}
        }

        # Use custom skills if provided, otherwise use default
        skills_db = custom_skills if custom_skills else self.technical_skills

        for category, skills in skills_db.items():
            found_in_category = []
            for skill in skills:
                # Use regex to match whole words to avoid partial matches (e.g., 'go' in 'golang')
                # But for multi-word skills, just check for presence
                if ' ' in skill:
                    if skill.lower() in text_lower:
                        found_in_category.append(skill)
                else:
                    if re.search(r'\b' + re.escape(skill.lower()) + r'\b', text_lower):
                        found_in_category.append(skill)

            # Add to main lists
            for skill in found_in_category:
                if skill not in found_skills['all_skills']:
                    found_skills['all_skills'].append(skill)

            if found_in_category:
                found_skills['by_category'][category] = list(set(found_in_category))

        found_skills['all_skills'] = list(set(found_skills['all_skills']))
        return found_skills

    def calculate_jaccard_similarity(self, set1, set2):
        """Calculate Jaccard Similarity: J(A,B) = |A ∩ B| / |A ∪ B|"""
        set1 = set([s.lower() for s in set1])
        set2 = set([s.lower() for s in set2])

        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))

        return intersection / union if union > 0 else 0.0

    def calculate_composite_score(self, jd_info, resume_info, cosine_score, weights=None):
        """
        Calculate composite score based on multiple factors, now including cosine similarity
        """
        if weights is None:
            # Added 'cosine' and re-balanced defaults
            weights = {
                'skills': 0.4,
                'cosine': 0.3,
                'experience': 0.2,
                'education': 0.1
            }

        # 1. Skills score (Jaccard similarity)
        skills_score = self.calculate_jaccard_similarity(
            jd_info['skills']['all_skills'],
            resume_info['skills']['all_skills']
        )

        # 2. Experience score
        exp_score = 0
        if 'required_experience' in jd_info and jd_info['required_experience'] > 0:
            # Cap at 1.0 (meeting requirement is 100%)
            exp_ratio = min(resume_info['experience']['total_years'] / jd_info['required_experience'], 1.0)
            exp_score = exp_ratio
        else:
            # If JD doesn't specify, score based on a 5-year benchmark
            exp_score = min(resume_info['experience']['total_years'] / 5.0, 1.0)

        # 3. Education score
        edu_score = 0
        degree_hierarchy = {'None': 0, 'Diploma': 1, 'Bachelors': 2, 'Masters': 3, 'Phd': 4}

        if 'required_education' in jd_info and jd_info['required_education'] != 'None':
            required_level = degree_hierarchy.get(jd_info['required_education'], 0)
            candidate_level = degree_hierarchy.get(resume_info['education']['highest_degree'], 0)

            if candidate_level >= required_level:
                edu_score = 1.0 # Meets or exceeds requirement
            elif required_level > 0:
                edu_score = candidate_level / required_level # Prorated score
            else:
                edu_score = 0.5 # No requirement specified, neutral score
        else:
            # If JD doesn't specify, score based on a Master's (3) benchmark
            edu_score = min(degree_hierarchy.get(resume_info['education']['highest_degree'], 0) / 3.0, 1.0)

        # 4. Cosine Score (already 0-1)
        # We use the passed-in cosine_score directly

        # Calculate weighted composite score
        composite_score = (
            weights['skills'] * skills_score +
            weights['cosine'] * cosine_score +
            weights['experience'] * exp_score +
            weights['education'] * edu_score
        )

        return {
            'composite_score': composite_score,
            'skills_score': skills_score,
            'cosine_score': cosine_score,
            'experience_score': exp_score,
            'education_score': edu_score
        }

    def parse_job_description(self, jd_text):
        """Parse job description to extract requirements"""
        doc = self.nlp(jd_text)

        jd_info = {
            'text': jd_text,
            'processed_text': self.preprocess_text(jd_text),
            'skills': self.extract_skills(jd_text),
            'experience': self.extract_experience(jd_text),
            'education': self.extract_education(jd_text, doc),
            'contact_info': self.extract_contact_info_and_name(jd_text, doc) # Use new function
        }

        jd_info['required_experience'] = jd_info['experience']['total_years']
        jd_info['required_education'] = jd_info['education']['highest_degree']

        return jd_info

    def screen_resumes(self, job_description, resume_folder, top_n=5, weights=None):
        """
        Screen resumes with advanced analysis
        """
        # Parse job description
        print("Parsing Job Description...")
        jd_info = self.parse_job_description(job_description)

        print(f"\n{'='*80}")
        print("JOB DESCRIPTION ANALYSIS")
        print(f"{'='*80}")
        print(f"Required Skills: {len(jd_info['skills']['all_skills'])}")
        print(f"Skills: {', '.join(jd_info['skills']['all_skills'][:10])}...")
        print(f"Required Experience: {jd_info['required_experience']} years")
        print(f"Required Education: {jd_info['required_education']}")
        print(f"{'='*80}\n")

        # Process all resumes
        results = []
        pdf_files = [f for f in os.listdir(resume_folder) if f.endswith('.pdf')]

        if not pdf_files:
            print("No PDF files found in the folder!")
            return pd.DataFrame(), jd_info, pd.DataFrame()

        print(f"Processing {len(pdf_files)} resumes...\n")

        # --- Prepare for Cosine Similarity ---
        resume_texts = []
        valid_pdf_files = [] # Keep track of files that were successfully read
        for pdf_file in pdf_files:
            pdf_path = os.path.join(resume_folder, pdf_file)
            resume_text = self.extract_text_from_pdf(pdf_path)
            if resume_text:
                resume_texts.append(resume_text)
                valid_pdf_files.append(pdf_file)
            else:
                 print(f"Skipping {pdf_file}: Could not extract text.")

        if not resume_texts:
            print("No resumes could be read.")
            return pd.DataFrame(), jd_info, pd.DataFrame()

        # Preprocess all texts
        jd_processed = jd_info['processed_text']
        resumes_processed = [self.preprocess_text(t) for t in resume_texts]

        # Fit and Transform with TF-IDF
        corpus = [jd_processed] + resumes_processed
        self.tfidf_vectorizer.fit(corpus)

        jd_vector = self.tfidf_vectorizer.transform([jd_processed])
        resumes_vector = self.tfidf_vectorizer.transform(resumes_processed)

        # Calculate Cosine Similarity for all resumes at once
        cosine_scores = cosine_similarity(jd_vector, resumes_vector)[0]

        # --- Loop and analyze each resume ---
        for idx, pdf_file in enumerate(valid_pdf_files):
            print(f"Analyzing {idx+1}/{len(valid_pdf_files)}: {pdf_file}")

            resume_text = resume_texts[idx]
            resume_processed = resumes_processed[idx]

            doc = self.nlp(resume_text)

            # Analyze resume
            resume_info = {
                'text': resume_text,
                'processed_text': resume_processed,
                'contact_info': self.extract_contact_info_and_name(resume_text, doc),
                'skills': self.extract_skills(resume_text),
                'experience': self.extract_experience(resume_text),
                'education': self.extract_education(resume_text, doc)
            }

            cosine_score = cosine_scores[idx]
            scores = self.calculate_composite_score(jd_info, resume_info, cosine_score, weights)

            matched_skills = list(set([s.lower() for s in jd_info['skills']['all_skills']]).intersection(
                set([s.lower() for s in resume_info['skills']['all_skills']])
            ))

            results.append({
                'Resume': pdf_file,
                'Candidate_Name': resume_info['contact_info']['name'],
                'Composite_Score': scores['composite_score'],
                'Cosine_Score': scores['cosine_score'],
                'Skills_Score': scores['skills_score'],
                'Experience_Score': scores['experience_score'],
                'Education_Score': scores['education_score'],
                'Total_Years_Experience': resume_info['experience']['total_years'],
                'Highest_Degree': resume_info['education']['highest_degree'],
                'Matched_Skills': len(matched_skills),
                'Total_JD_Skills': len(jd_info['skills']['all_skills']),
                'Total_Resume_Skills': len(resume_info['skills']['all_skills']),
                'Skills_List': ', '.join(matched_skills),
                'Match_Percentage': (len(matched_skills) / len(jd_info['skills']['all_skills']) * 100) if len(jd_info['skills']['all_skills']) > 0 else 0,
                'Email': resume_info['contact_info']['email'],
                'Phone': resume_info['contact_info']['phone'],
                'Skills_By_Category': json.dumps(resume_info['skills']['by_category'])
            })

        # Create DataFrame and sort
        df_results = pd.DataFrame(results)
        df_results = df_results.sort_values('Composite_Score', ascending=False).reset_index(drop=True)

        return df_results.head(top_n), jd_info, df_results # Return all results for visualization

    def visualize_results(self, all_results_df, jd_info, output_folder='visualizations'):
        """
        Create comprehensive visualizations of screening results
        """
        if all_results_df.empty:
            print("No results to visualize!")
            return

        # Use top 15 for visualization to keep charts readable
        results_df = all_results_df.head(15)

        os.makedirs(output_folder, exist_ok=True)
        sns.set_style("whitegrid")
        plt.rcParams['figure.figsize'] = (12, 8)

        # 1. Composite Score Comparison (Top N)
        fig, ax = plt.subplots(figsize=(14, max(8, len(results_df) * 0.5)))
        candidates = [f"{row['Candidate_Name']}\n({row['Resume']})" for _, row in results_df.iterrows()]
        scores = results_df['Composite_Score'].values

        colors = sns.color_palette("RdYlGn", len(candidates))
        bars = ax.barh(candidates, scores, color=colors)
        ax.set_xlabel('Composite Score', fontsize=12, fontweight='bold')
        ax.set_title(f'Top {len(results_df)} Candidates - Composite Score Ranking', fontsize=14, fontweight='bold', pad=20)
        ax.set_xlim(0, 1)
        ax.invert_yaxis() # Show rank #1 at the top

        for i, (bar, score) in enumerate(zip(bars, scores)):
            ax.text(score + 0.01, bar.get_y() + bar.get_height()/2,
                   f'{score:.3f}', va='center', fontsize=10, fontweight='bold')

        plt.tight_layout()
        plt.savefig(f'{output_folder}/1_composite_scores.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/1_composite_scores.png")
        plt.close()

        # 2. Multi-dimensional Score Breakdown (Top N)
        fig, ax = plt.subplots(figsize=(14, 8))

        # Melt the dataframe for easier plotting with seaborn
        df_melted = results_df.melt(
            id_vars='Candidate_Name',
            value_vars=['Skills_Score', 'Cosine_Score', 'Experience_Score', 'Education_Score'],
            var_name='Score_Type',
            value_name='Score'
        )

        sns.barplot(x='Candidate_Name', y='Score', hue='Score_Type', data=df_melted, ax=ax, palette='Set2')
        ax.set_xlabel('Candidates', fontsize=12, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12, fontweight='bold')
        ax.set_title('Score Breakdown by Dimension', fontsize=14, fontweight='bold', pad=20)

        # --- THIS IS THE FIX for the UserWarning ---
        # We set the rotation on the tick parameters
        ax.tick_params(axis='x', rotation=45)
        # Manually adjust layout to prevent labels from being cut off
        plt.setp(ax.get_xticklabels(), ha='right')

        ax.legend(fontsize=10)
        ax.set_ylim(0, 1)

        plt.tight_layout()
        plt.savefig(f'{output_folder}/2_score_breakdown.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/2_score_breakdown.png")
        plt.close()

        # 3. Skills Match Visualization (Top N)
        fig, ax = plt.subplots(figsize=(14, max(8, len(results_df) * 0.5)))
        candidates_short = [row['Candidate_Name'] for _, row in results_df.iterrows()]
        matched = results_df['Matched_Skills'].values
        total_jd = results_df['Total_JD_Skills'].values[0] if len(results_df) > 0 else 0
        unmatched = [total_jd - m for m in matched]

        bars1 = ax.barh(candidates_short, matched, label='Matched Skills', color='#27ae60')
        bars2 = ax.barh(candidates_short, unmatched, left=matched, label='Unmatched Skills (JD)', color='#ecf0f1')
        ax.set_xlabel('Number of Skills', fontsize=12, fontweight='bold')
        ax.set_title(f'Skills Match Analysis (Total Required: {total_jd})', fontsize=14, fontweight='bold', pad=20)
        ax.legend(fontsize=10)
        ax.invert_yaxis()

        for i, (m, u) in enumerate(zip(matched, unmatched)):
            percentage = (m / total_jd * 100) if total_jd > 0 else 0
            ax.text(m/2, i, f'{m}', ha='center', va='center', fontweight='bold', color='white')
            ax.text(m + u + 0.5, i, f'{percentage:.1f}%', ha='left', va='center', fontweight='bold')

        plt.tight_layout()
        plt.savefig(f'{output_folder}/3_skills_match.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/3_skills_match.png")
        plt.close()

        # 4. Experience vs Score Scatter (All Candidates)
        fig, ax = plt.subplots(figsize=(12, 8))
        scatter = ax.scatter(all_results_df['Total_Years_Experience'],
                             all_results_df['Composite_Score'],
                             s=100, alpha=0.6, c=all_results_df['Composite_Score'],
                             cmap='viridis', edgecolors='black', linewidth=1)

        # Annotate top 5
        for idx, row in all_results_df.head(5).iterrows():
            ax.annotate(row['Candidate_Name'],
                       (row['Total_Years_Experience'], row['Composite_Score']),
                       xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

        # Add JD requirement line
        if jd_info['required_experience'] > 0:
            ax.axvline(x=jd_info['required_experience'], color='red', linestyle='--', label=f'JD Req: {jd_info["required_experience"]} yrs')
            ax.legend()

        ax.set_xlabel('Years of Experience', fontsize=12, fontweight='bold')
        ax.set_ylabel('Composite Score', fontsize=12, fontweight='bold')
        ax.set_title('Experience vs Composite Score (All Candidates)', fontsize=14, fontweight='bold', pad=20)
        plt.colorbar(scatter, ax=ax, label='Composite Score')
        plt.tight_layout()
        plt.savefig(f'{output_folder}/4_experience_vs_score.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/4_experience_vs_score.png")
        plt.close()

        # 5. Education Distribution (All Candidates)
        fig, ax = plt.subplots(figsize=(10, 8))
        edu_counts = all_results_df['Highest_Degree'].value_counts()
        colors_pie = sns.color_palette("Set2", len(edu_counts))

        wedges, texts, autotexts = ax.pie(edu_counts.values, labels=edu_counts.index,
                                          autopct='%1.1f%%', startangle=90,
                                          colors=colors_pie, textprops={'fontsize': 11, 'fontweight': 'bold'})
        ax.set_title('Education Level Distribution (All Candidates)', fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(f'{output_folder}/5_education_distribution.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/5_education_distribution.png")
        plt.close()

        # 6. Radar Chart for Top 3 Candidates
        if len(results_df) >= 3:
            fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

            categories = ['Skills', 'Semantic Match', 'Experience', 'Education']
            num_vars = len(categories)

            angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
            angles += angles[:1]

            ax.set_theta_offset(np.pi / 2)
            ax.set_theta_direction(-1)
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(categories, fontsize=12, fontweight='bold')

            colors_radar = ['#3498db', '#e74c3c', '#2ecc71']

            for idx in range(min(3, len(results_df))):
                row = results_df.iloc[idx]
                values = [row['Skills_Score'], row['Cosine_Score'], row['Experience_Score'], row['Education_Score']]
                values += values[:1]

                ax.plot(angles, values, 'o-', linewidth=2, label=row['Candidate_Name'], color=colors_radar[idx])
                ax.fill(angles, values, alpha=0.15, color=colors_radar[idx])

            ax.set_ylim(0, 1)
            ax.set_title('Top 3 Candidates - Profile Radar', fontsize=14, fontweight='bold', pad=30)
            ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)

            plt.tight_layout()
            plt.savefig(f'{output_folder}/6_radar_chart.png', dpi=300, bbox_inches='tight')
            print(f"✓ Saved: {output_folder}/6_radar_chart.png")
            plt.close()

        print(f"\n✓ All visualizations saved in '{output_folder}' folder!")

    def display_results(self, results_df, jd_info):
        """Display detailed screening results"""
        if results_df.empty:
            print("No results to display!")
            return

        print("\n" + "="*100)
        print(f"{'TOP ' + str(len(results_df)) + ' SHORTLISTED CANDIDATES':^100}")
        print("="*100)

        for idx, row in results_df.iterrows():
            rank = idx + 1
            print(f"\n{'RANK #' + str(rank):^100}")
            print("-" * 100)
            print(f"{'Candidate:':<25} {row['Candidate_Name']}")
            print(f"{'Resume File:':<25} {row['Resume']}")
            print(f"{'Email:':<25} {row['Email']}")
            print(f"{'Phone:':<25} {row['Phone']}")
            print()
            print(f"{'SCORES:':<25}")
            print(f"{'  Composite Score:':<25} {row['Composite_Score']:.4f} ({row['Composite_Score']*100:.1f}%)")
            print(f"{'  Skills (Jaccard) Score:':<25} {row['Skills_Score']:.4f} ({row['Skills_Score']*100:.1f}%)")
            print(f"{'  Semantic (Cosine) Score:':<25} {row['Cosine_Score']:.4f} ({row['Cosine_Score']*100:.1f}%)")
            print(f"{'  Experience Score:':<25} {row['Experience_Score']:.4f} ({row['Experience_Score']*100:.1f}%)")
            print(f"{'  Education Score:':<25} {row['Education_Score']:.4f} ({row['Education_Score']*100:.1f}%)")
            print()
            print(f"{'QUALIFICATIONS:':<25}")
            print(f"{'  Experience:':<25} {row['Total_Years_Experience']} years (JD: {jd_info['required_experience']} yrs)")
            print(f"{'  Education:':<25} {row['Highest_Degree']} (JD: {jd_info['required_education']})")
            print(f"{'  Matched Skills:':<25} {row['Matched_Skills']} of {row['Total_JD_Skills']} ({row['Match_Percentage']:.1f}%)")
            print(f"{'  Total Skills:':<25} {row['Total_Resume_Skills']}")
            print()
            print(f"{'MATCHED SKILLS:':<25}")
            skills_list = row['Skills_List'].split(', ') if row['Skills_List'] else ["None"]
            for i in range(0, len(skills_list), 5):
                print(f"{'  ':<25} {', '.join(skills_list[i:i+5])}")

        print("\n" + "="*100 + "\n")

    def save_results(self, results_df, output_file='screening_results.csv'):
        """Save detailed results to CSV"""
        results_df.to_csv(output_file, index=False)
        print(f"✓ Results saved to {output_file}")

    def generate_report(self, results_df, jd_info, all_results_df, output_file='screening_report.html'):
        """Generate HTML report with all results"""
        html_content = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>Resume Screening Report</title>
            <style>
                body {{
                    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                    margin: 40px;
                    background-color: #f5f5f5;
                }}
                .header {{
                    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white;
                    padding: 30px;
                    border-radius: 10px;
                    margin-bottom: 30px;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.1);
                }}
                .header h1 {{ margin: 0; font-size: 32px; }}
                .jd-section {{
                    background: white;
                    padding: 25px;
                    border-radius: 10px;
                    margin-bottom: 30px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                }}
                .candidate {{
                    background: white;
                    padding: 25px;
                    margin-bottom: 20px;
                    border-radius: 10px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    border-left: 5px solid #667eea;
                }}
                .rank {{
                    display: inline-block;
                    background: #667eea;
                    color: white;
                    padding: 8px 15px;
                    border-radius: 20px;
                    font-weight: bold;
                    font-size: 18px;
                }}
                .score-container {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(120px, 1fr));
                    gap: 15px;
                    margin: 20px 0;
                }}
                .score-box {{
                    background: #f8f9fa;
                    padding: 15px;
                    border-radius: 8px;
                    text-align: center;
                    border-bottom: 3px solid #667eea;
                }}
                .score-value {{
                    font-size: 28px;
                    font-weight: bold;
                    color: #667eea;
                }}
                .score-label {{ color: #666; font-size: 14px; margin-top: 5px; }}
                .skills {{ display: flex; flex-wrap: wrap; gap: 8px; margin-top: 15px; }}
                .skill-tag {{
                    background: #e3f2fd;
                    color: #1976d2;
                    padding: 5px 12px;
                    border-radius: 15px;
                    font-size: 13px;
                }}
                .info-row {{
                    display: flex;
                    justify-content: space-between;
                    padding: 10px 0;
                    border-bottom: 1px solid #eee;
                }}
                .info-label {{ font-weight: bold; color: #555; }}
            </style>
        </head>
        <body>
            <div class="header">
                <h1>📋 Resume Screening Report</h1>
                <p>Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            </div>

            <div class="jd-section">
                <h2>📄 Job Description Analysis</h2>
                <div class="info-row">
                    <span class="info-label">Total Required Skills:</span>
                    <span>{len(jd_info['skills']['all_skills'])}</span>
                </div>
                <div class="info-row">
                    <span class="info-label">Required Experience:</span>
                    <span>{jd_info['required_experience']} years</span>
                </div>
                <div class="info-row">
                    <span class="info-label">Required Education:</span>
                    <span>{jd_info['required_education']}</span>
                </div>
                <div class="info-row">
                    <span class="info-label">Total Candidates Screened:</span>
                    <span>{len(all_results_df)}</span>
                </div>
            </div>
        """

        for idx, row in results_df.iterrows():
            rank = idx + 1
            skills_list = row['Skills_List'].split(', ') if row['Skills_List'] else []
            skills_html = ''.join([f'<span class="skill-tag">{skill}</span>' for skill in skills_list])
            if not skills_html: skills_html = "<span>No matched skills found.</span>"

            html_content += f"""
            <div class="candidate">
                <span class="rank">Rank #{rank}</span>
                <h2 style="margin-top: 15px;">{row['Candidate_Name']}</h2>
                <p style="color: #666;">{row['Resume']}</p>

                <div class="score-container">
                    <div class="score-box">
                        <div class="score-value">{row['Composite_Score']:.3f}</div>
                        <div class="score-label">Composite</div>
                    </div>
                    <div class="score-box">
                        <div class="score-value">{row['Skills_Score']:.3f}</div>
                        <div class="score-label">Skills (Jaccard)</div>
                    </div>
                    <div class="score-box">
                        <div class="score-value">{row['Cosine_Score']:.3f}</div>
                        <div class="score-label">Semantic (Cosine)</div>
                    </div>
                    <div class="score-box">
                        <div class="score-value">{row['Experience_Score']:.3f}</div>
                        <div class="score-label">Experience</div>
                    </div>
                    <div class="score-box">
                        <div class="score-value">{row['Education_Score']:.3f}</div>
                        <div class="score-label">Education</div>
                    </div>
                </div>

                <div class="info-row">
                    <span class="info-label">📧 Email:</span>
                    <span>{row['Email']}</span>
                </div>
                <div class="info-row">
                    <span class="info-label">📱 Phone:</span>
                    <span>{row['Phone']}</span>
                </div>
                <div class="info-row">
                    <span class="info-label">💼 Experience:</span>
                    <span>{row['Total_Years_Experience']} years</span>
                </div>
                <div class="info-row">
                    <span class="info-label">🎓 Education:</span>
                    <span>{row['Highest_Degree']}</span>
                </div>
                <div class="info-row">
                    <span class="info-label">✅ Skills Match:</span>
                    <span>{row['Matched_Skills']} of {row['Total_JD_Skills']} ({row['Match_Percentage']:.1f}%)</span>
                </div>

                <h3 style="margin-top: 20px;">Matched Skills:</h3>
                <div class="skills">
                    {skills_html}
                </div>
            </div>
            """

        html_content += """
        </body>
        </html>
        """

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(html_content)

        print(f"✓ HTML report saved to {output_file}")


def main():
    """Main function to run the advanced resume screening system"""
    print("="*100)
    print(f"{'ADVANCED RESUME SCREENING SYSTEM':^100}")
    print(f"{'with TF-IDF, NER, Experience Extraction & Visualization':^100}")
    print("="*100)
    print()

    # Initialize screener
    # --- Make sure to use the correct path for your Colab environment ---
    screener = AdvancedResumeScreener(skills_db_path='/content/drive/MyDrive/CDS_PROJECT/skills_db.json')

    # Example job description
    default_jd = """
    Senior Software Engineer - Machine Learning

    We are seeking a talented Senior Software Engineer with 5+ years of experience to join our AI team.

    Required Skills:
    - Strong programming skills in Python and Java
    - Experience with Machine Learning frameworks (TensorFlow, PyTorch, scikit-learn)
    - Deep Learning and Neural Networks
    - SQL and Database management (MySQL, PostgreSQL)
    - REST API development and microservices architecture
    - Cloud platforms (AWS, Azure, or GCP)
    - Git version control and CI/CD pipelines
    - Docker and Kubernetes

    Required Qualifications:
    - Bachelor's degree in Computer Science or related field (Master's preferred)
    - 5+ years of professional software development experience
    - Strong problem-solving and analytical skills
    - Excellent communication and teamwork abilities

    Preferred:
    - Experience with NLP and Computer Vision
    - Knowledge of Agile/Scrum methodologies
    - Contributions to open-source projects
    """

    # Get inputs
    print("=" * 100)
    print("STEP 1: JOB DESCRIPTION")
    print("=" * 100)
    print("\nOptions:")
    print("1. Use default job description (Senior Software Engineer - ML)")
    print("2. Enter custom job description")
    print("3. Load from file")

    choice = input("\nEnter your choice (1/2/3) [default: 1]: ").strip()

    if choice == '2':
        print("\nEnter job description (press Ctrl+D on Linux/macOS or Ctrl+Z+Enter on Windows when done):")
        job_description = ""
        try:
            while True:
                line = input()
                job_description += line + "\n"
        except EOFError:
            pass
    elif choice == '3':
        jd_file = input("Enter path to job description file: ").strip()
        try:
            with open(jd_file, 'r', encoding='utf-8') as f:
                job_description = f.read()
        except:
            print("Error reading file. Using default JD.")
            job_description = default_jd
    else:
        job_description = default_jd

    # Get resume folder
    print("\n" + "=" * 100)
    print("STEP 2: RESUME FOLDER")
    print("=" * 100)
    resume_folder = input("\nEnter path to resume folder [default: ./resumes]: ").strip()
    if not resume_folder:
        resume_folder = "./resumes"

    if not os.path.exists(resume_folder):
        print(f"\nFolder '{resume_folder}' not found!")
        create = input("Create folder? (y/n): ").strip().lower()
        if create == 'y':
            os.makedirs(resume_folder)
            print(f"✓ Folder created. Please add PDF resumes and run again.")
            return
        else:
            return

    # Get top N
    print("\n" + "=" * 100)
    print("STEP 3: SCREENING PARAMETERS")
    print("=" * 100)
    top_n_input = input("\nEnter number of top candidates to shortlist [default: 5]: ").strip()
    top_n = int(top_n_input) if top_n_input.isdigit() and int(top_n_input) > 0 else 5

    # Get scoring weights
    print("\nUse custom scoring weights? (y/n) [default: n]: ", end='')
    custom_weights = input().strip().lower()

    weights = None
    if custom_weights == 'y':
        print("\nEnter weights (must sum to 1.0):")
        try:
            skills_w = float(input("  Skills (Jaccard) weight [default: 0.4]: ") or 0.4)
            cosine_w = float(input("  Semantic (Cosine) weight [default: 0.3]: ") or 0.3)
            exp_w = float(input("  Experience weight [default: 0.2]: ") or 0.2)
            edu_w = float(input("  Education weight [default: 0.1]: ") or 0.1)

            if abs(skills_w + cosine_w + exp_w + edu_w - 1.0) < 0.01:
                weights = {'skills': skills_w, 'cosine': cosine_w, 'experience': exp_w, 'education': edu_w}
            else:
                print("Weights don't sum to 1.0. Using defaults.")
        except:
            print("Invalid input. Using default weights.")

    # Process resumes
    print("\n" + "=" * 100)
    print("PROCESSING RESUMES...")
    print("=" * 100)

    # Suppress warnings during visualization
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        top_results, jd_info, all_results = screener.screen_resumes(job_description, resume_folder, top_n, weights)

    if top_results.empty:
        print("\n❌ No resumes could be processed!")
        return

    # Display results
    screener.display_results(top_results, jd_info)

    # Save results
    print("\n" + "=" * 100)
    print("SAVING RESULTS...")
    print("=" * 100)
    screener.save_results(all_results, 'screening_results.csv') # Save all results

    # Generate HTML report
    screener.generate_report(top_results, jd_info, all_results, 'screening_report.html')

    # Generate visualizations
    print("\n" + "=" * 100)
    print("GENERATING VISUALIZATIONS...")
    print("=" * 100)
    visualize = input("\nGenerate visualizations? (y/n) [default: y]: ").strip().lower()

    if visualize != 'n':
        # Suppress warnings during visualization
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            screener.visualize_results(all_results, jd_info, 'visualizations')

    print("\n" + "="*100)
    print(f"{'✓ SCREENING COMPLETED SUCCESSFULLY!':^100}")
    print("="*100)
    print("\nGenerated Files:")
    print("  📊 screening_results.csv - Detailed results for ALL candidates")
    print("  📄 screening_report.html - Interactive report of TOP candidates")
    if visualize != 'n':
        print("  📈 visualizations/ - Folder containing all charts and graphs")
    print("\n" + "="*100)


if __name__ == "__main__":
    main()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


                                  ADVANCED RESUME SCREENING SYSTEM                                  
                      with TF-IDF, NER, Experience Extraction & Visualization                       

Loading skills database from /content/drive/MyDrive/CDS_PROJECT/skills_db.json...
STEP 1: JOB DESCRIPTION

Options:
1. Use default job description (Senior Software Engineer - ML)
2. Enter custom job description
3. Load from file

Enter your choice (1/2/3) [default: 1]: 1

STEP 2: RESUME FOLDER

Enter path to resume folder [default: ./resumes]: /content/drive/MyDrive/CDS_PROJECT/Resumes

STEP 3: SCREENING PARAMETERS

Enter number of top candidates to shortlist [default: 5]: 4

Use custom scoring weights? (y/n) [default: n]: n

PROCESSING RESUMES...
Parsing Job Description...

JOB DESCRIPTION ANALYSIS
Required Skills: 25
Skills: git, ci/cd, java, python, communication, aws, postgresql, sql, computer vision, analytical...
Required Experience: 5 years
Required Education: Masters

Processing

# **NEW**

In [ ]:
"""
Advanced Resume Screening and Shortlisting System (Upgraded v3 - Academic Evaluation)

Features:
- NLP, NER, TF-IDF, Cosine Similarity
- Robust Experience/Name Extraction
- Visualization
- Scoring formula based on academic paper (0.6 * Cosine + 0.4 * Jaccard)
- Skill extraction accuracy evaluation function
"""

import os
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import pdfplumber
import logging
import warnings # To handle the matplotlib warning

# Suppress noisy logs from pdfminer (used by pdfplumber)
logging.getLogger('pdfminer').setLevel(logging.ERROR)

# --- NLTK Data Downloads ---
def download_nltk_data():
    """Download required NLTK data models if not found."""
    nltk_packages = [
        ('tokenizers/punkt', 'punkt'),
        ('tokenizers/punkt_tab', 'punkt_tab'),
        ('corpora/stopwords', 'stopwords'),
        ('corpora/wordnet', 'wordnet'),
        ('corpora/omw-1.4', 'omw-1.4')
    ]
    for path, package_id in nltk_packages:
        try:
            nltk.data.find(path)
        except LookupError:
            print(f"Downloading NLTK package: {package_id}...")
            nltk.download(package_id)

download_nltk_data()

# --- Load spaCy model for NER ---
def load_spacy_model(model_name="en_core_web_sm"):
    """Load or download spaCy model."""
    try:
        nlp = spacy.load(model_name)
    except:
        print(f"Downloading spaCy model: {model_name}...")
        os.system(f"python -m spacy download {model_name}")
        nlp = spacy.load(model_name)
    return nlp

nlp = load_spacy_model()


class AdvancedResumeScreener:
    """
    Advanced Resume Screening System with NER, TF-IDF, Experience, Education, and Visualization
    """

    def __init__(self, skills_db_path='skills_db.json'):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english')) - {'and', 'to', 'of'}
        self.nlp = nlp
        self.matcher = Matcher(self.nlp.vocab)

        # Blacklist for common non-name headers
        self.NAME_BLACKLIST = {
            'linkedin', 'github', 'portfolio', 'profile', 'leetcode', 'email', 'phone',
            'address', 'resume', 'cv', 'work', 'contact', 'experience', 'education',
            'skills', 'projects', 'summary'
        }

        # Education degrees
        self.education_keywords = {
            'phd': ['phd', 'ph.d', 'doctorate', 'doctoral'],
            'masters': ['masters', 'master', 'msc', 'm.sc', 'ma', 'm.a', 'mba', 'm.b.a', 'mtech', 'm.tech'],
            'bachelors': ['bachelors', 'bachelor', 'bsc', 'b.sc', 'ba', 'b.a', 'btech', 'b.tech', 'be', 'b.e'],
            'diploma': ['diploma', 'associate']
        }

        # Load skills from external JSON
        self.technical_skills = self.load_skills_from_json(skills_db_path)

        # Initialize TF-IDF Vectorizer
        self.tfidf_vectorizer = TfidfVectorizer(
            stop_words='english',
            ngram_range=(1, 2)  # Capture single words and two-word phrases
        )

    def load_skills_from_json(self, json_path):
        """Loads the skills database from an external JSON file."""
        try:
            with open(json_path, 'r') as f:
                print(f"Loading skills database from {json_path}...")
                return json.load(f)
        except FileNotFoundError:
            print(f"Error: {json_path} not found. Using empty skills DB.")
            return {}
        except json.JSONDecodeError:
            print(f"Error: Could not decode {json_path}. Using empty skills DB.")
            return {}

    def extract_text_from_pdf(self, pdf_path):
        """Extract text from PDF resume using pdfplumber for better layout handling"""
        text = ""
        try:
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    # Extract text, respecting layout
                    page_text = page.extract_text(x_tolerance=2, y_tolerance=2)
                    if page_text:
                        text += page_text + "\n"
                return text
        except Exception as e:
            print(f"Error reading {pdf_path} with pdfplumber: {str(e)}")
            return ""  # Return empty string on failure

    def preprocess_text(self, text):
        """Preprocess text with cleaning, tokenization, and lemmatization"""
        text = text.lower()
        text = re.sub(r'\\x[a-f0-9]{2}', '', text)  # Remove hex characters
        text = re.sub(r'\\[abtnr]', '', text)     # Remove escape sequences
        text = re.sub(r'[^\w\s]', ' ', text)       # Remove punctuation
        text = re.sub(r'\s+', ' ', text)           # Condense whitespace

        tokens = word_tokenize(text)
        tokens = [token for token in tokens if token not in self.stop_words and len(token) > 1]
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]

        return ' '.join(tokens)

    def extract_contact_info_and_name(self, text, doc):
        """
        Extracts contact info using regex and a robust, multi-step name extraction logic
        """
        info = {
            'name': 'Unknown',
            'email': 'N/A',
            'phone': 'N/A'
        }

        # --- 1. Extract Name (More Robust Multi-Step Logic) ---

        # Step 1: High-Confidence Search
        # Look for a 'PERSON' entity in the first 75 tokens (the top of the resume)
        person_entities = [ent.text for ent in doc[:75].ents if ent.label_ == 'PERSON']
        if person_entities:
            info['name'] = person_entities[0] # Take the first one

        # Step 2: Blacklist Check
        # Check if the found name is likely a header (e.g., "LinkedIn")
        name_parts = set(info['name'].lower().split())
        if info['name'] == 'Unknown' or name_parts.intersection(self.NAME_BLACKLIST):
            # Name is "Unknown" OR it's a blacklisted word, so we reset and try Fallback 1
            info['name'] = 'Unknown'

            # Fallback 1: Try the Proper Noun (PROPN) Matcher
            pattern1 = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
            pattern2 = [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]

            if "NAME_PATTERN_1" not in self.matcher._patterns:
                self.matcher.add("NAME_PATTERN_1", [pattern1], on_match=None)
            if "NAME_PATTERN_2" not in self.matcher._patterns:
                self.matcher.add("NAME_PATTERN_2", [pattern2], on_match=None)

            matches = self.matcher(doc[:75])
            if matches:
                matches.sort(key=lambda x: x[2] - x[1], reverse=True) # Get longest match
                match_id, start, end = matches[0]
                potential_name = doc[start:end].text

                # Check blacklist *again*
                potential_name_parts = set(potential_name.lower().split())
                if not potential_name_parts.intersection(self.NAME_BLACKLIST):
                    info['name'] = potential_name

        # Fallback 2: Last Resort
        # If name is *still* Unknown, search the *entire document* for the first PERSON entity
        if info['name'] == 'Unknown':
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    name_parts = set(ent.text.lower().split())
                    if not name_parts.intersection(self.NAME_BLACKLIST):
                        info['name'] = ent.text
                        break  # Take the first valid one

        # --- 2. Extract Email ---
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        email_match = re.search(email_pattern, text)
        if email_match:
            info['email'] = email_match.group(0)

        # --- 3. Extract Phone ---
        phone_pattern = r'(\(?\+?\d{1,3}\)?[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?[\d\s-]{10,15}\b'
        phone_match = re.search(phone_pattern, text)
        if phone_match:
            info['phone'] = phone_match.group(0).strip()

        return info

    def extract_experience(self, text):
        """
        Extract years of experience from resume by parsing date ranges.
        """
        date_range_pattern = re.compile(
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w.]{0,6}\s+\d{4}|'  # Month YYYY
            r'\d{1,2}[\/.-]\d{4}|'  # MM/YYYY
            r'\b\d{4}\b)'  # YYYY
            r'\s*(?:[-–—toTO])\s*'
            r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\w.]{0,6}\s+\d{4}|'  # Month YYYY
            r'\d{1,2}[\/.-]\d{4}|'  # MM/YYYY
            r'\b\d{4}\b|'  # YYYY
            r'Present|Current|Till Date|Now)',  # Present
            re.IGNORECASE
        )
        matches = date_range_pattern.findall(text)
        if not matches:
            exp_patterns = r'(\d+)\+?\s*(?:years?|yrs?)\s*(?:of)?\s*experience'
            exp_match = re.search(exp_patterns, text.lower())
            if exp_match:
                return {'total_years': int(exp_match.group(1))}
            return {'total_years': 0}

        month_map = {
            'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
            'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
        }
        def parse_date(date_str):
            date_str = date_str.lower().strip().replace('.', '')
            if date_str in ['present', 'current', 'till date', 'now']:
                return datetime.now()
            for month_name, month_num in month_map.items():
                if date_str.startswith(month_name):
                    year_match = re.search(r'(\d{4})', date_str)
                    if year_match:
                        return datetime(int(year_match.group(1)), month_num, 1)
            match = re.match(r'(\d{1,2})[\/.-](\d{4})', date_str)
            if match:
                return datetime(int(match.group(2)), int(match.group(1)), 1)
            match = re.match(r'^(\d{4})$', date_str)
            if match:
                return datetime(int(match.group(1)), 1, 1)
            return None

        parsed_ranges = []
        for start_str, end_str in matches:
            start_date = parse_date(start_str)
            end_date = parse_date(end_str)
            if start_date and end_date:
                parsed_ranges.append((start_date, end_date))
        if not parsed_ranges:
            return {'total_years': 0}

        parsed_ranges.sort(key=lambda x: x[0])
        merged_ranges = []
        if parsed_ranges:
            merged_ranges = [parsed_ranges[0]]
            for current_start, current_end in parsed_ranges[1:]:
                last_start, last_end = merged_ranges[-1]
                if current_start <= last_end:
                    new_end = max(last_end, current_end)
                    merged_ranges[-1] = (last_start, new_end)
                else:
                    merged_ranges.append((current_start, current_end))

        total_months = 0
        for start_date, end_date in merged_ranges:
            months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
            total_months += (months + 1)

        return {'total_years': round(total_months / 12, 1)}

    def extract_education(self, text, doc):
        """Extract education qualifications from resume"""
        education_info = {
            'highest_degree': 'None',
            'degrees': [],
            'institutions': []
        }
        text_lower = text.lower()
        degree_priority = ['phd', 'masters', 'bachelors', 'diploma']
        for degree in degree_priority:
            for keyword in self.education_keywords[degree]:
                if keyword in text_lower:
                    if education_info['highest_degree'] == 'None':
                        education_info['highest_degree'] = degree.title()
                    if degree.title() not in education_info['degrees']:
                        education_info['degrees'].append(degree.title())
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                org_lower = ent.text.lower()
                if any(word in org_lower for word in ['university', 'college', 'institute', 'school']):
                    if ent.text not in education_info['institutions']:
                        education_info['institutions'].append(ent.text)
        education_info['institutions'] = list(set(education_info['institutions']))
        return education_info

    def extract_skills(self, text, custom_skills=None):
        """Extract skills from text with categorization"""
        text_lower = text.lower()
        found_skills = {'all_skills': [], 'by_category': {}}
        skills_db = custom_skills if custom_skills else self.technical_skills
        for category, skills in skills_db.items():
            found_in_category = []
            for skill in skills:
                if ' ' in skill:
                    if skill.lower() in text_lower:
                        found_in_category.append(skill)
                else:
                    if re.search(r'\b' + re.escape(skill.lower()) + r'\b', text_lower):
                        found_in_category.append(skill)
            for skill in found_in_category:
                if skill not in found_skills['all_skills']:
                    found_skills['all_skills'].append(skill)
            if found_in_category:
                found_skills['by_category'][category] = list(set(found_in_category))
        found_skills['all_skills'] = list(set(found_skills['all_skills']))
        return found_skills

    def calculate_jaccard_similarity(self, set1, set2):
        """Calculate Jaccard Similarity: J(A,B) = |A ∩ B| / |A ∪ B|"""
        set1 = set([s.lower() for s in set1])
        set2 = set([s.lower() for s in set2])
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        return intersection / union if union > 0 else 0.0

    def calculate_composite_score(self, jd_info, resume_info, cosine_score, weights):
        """
        Calculate composite score based on the flexible weights provided.
        Also calculates individual scores for reporting.
        """

        # --- 1. Calculate Jaccard Skill Score (for scoring) ---
        skills_score = self.calculate_jaccard_similarity(
            jd_info['skills']['all_skills'],
            resume_info['skills']['all_skills']
        )

        # --- 2. Calculate Experience Score (for reporting or scoring) ---
        exp_score = 0
        if 'required_experience' in jd_info and jd_info['required_experience'] > 0:
            exp_ratio = min(resume_info['experience']['total_years'] / jd_info['required_experience'], 1.0)
            exp_score = exp_ratio
        else:
            exp_score = min(resume_info['experience']['total_years'] / 5.0, 1.0)

        # --- 3. Calculate Education Score (for reporting or scoring) ---
        edu_score = 0
        degree_hierarchy = {'None': 0, 'Diploma': 1, 'Bachelors': 2, 'Masters': 3, 'Phd': 4}
        if 'required_education' in jd_info and jd_info['required_education'] != 'None':
            required_level = degree_hierarchy.get(jd_info['required_education'], 0)
            candidate_level = degree_hierarchy.get(resume_info['education']['highest_degree'], 0)
            if candidate_level >= required_level:
                edu_score = 1.0
            elif required_level > 0:
                edu_score = candidate_level / required_level
            else:
                edu_score = 0.5
        else:
            edu_score = min(degree_hierarchy.get(resume_info['education']['highest_degree'], 0) / 3.0, 1.0)

        # --- 4. Calculate Final Hybrid Score ---
        # This dynamically calculates the score based on the weights dict passed from main()
        # If a weight isn't in the dict (e.g., 'experience'), it defaults to 0 and adds nothing.
        final_score = (
            weights.get('cosine', 0) * cosine_score +
            weights.get('skills', 0) * skills_score +
            weights.get('experience', 0) * exp_score +
            weights.get('education', 0) * edu_score
        )

        return {
            'composite_score': final_score,
            'skills_score': skills_score,      # This is the Jaccard Score
            'cosine_score': cosine_score,      # This is the TF-IDF Cosine Score
            'experience_score': exp_score,    # For reporting
            'education_score': edu_score      # For reporting
        }

    def parse_job_description(self, jd_text):
        """Parse job description to extract requirements"""
        doc = self.nlp(jd_text)
        jd_info = {
            'text': jd_text,
            'processed_text': self.preprocess_text(jd_text),
            'skills': self.extract_skills(jd_text),
            'experience': self.extract_experience(jd_text),
            'education': self.extract_education(jd_text, doc),
            'contact_info': self.extract_contact_info_and_name(jd_text, doc)
        }
        jd_info['required_experience'] = jd_info['experience']['total_years']
        jd_info['required_education'] = jd_info['education']['highest_degree']
        return jd_info

    def screen_resumes(self, job_description, resume_folder, top_n=5, weights=None):
        """
        Screen resumes with advanced analysis.
        Uses the 'weights' dict passed from main() for scoring.
        """
        # Default to paper's formula if no weights are provided
        if weights is None:
            weights = {'cosine': 0.6, 'skills': 0.4}

        # Parse job description
        print("Parsing Job Description...")
        jd_info = self.parse_job_description(job_description)

        print(f"\n{'='*80}")
        print("JOB DESCRIPTION ANALYSIS")
        print(f"{'='*80}")
        print(f"Required Skills: {len(jd_info['skills']['all_skills'])}")
        print(f"Skills: {', '.join(jd_info['skills']['all_skills'][:10])}...")
        print(f"Required Experience: {jd_info['required_experience']} years")
        print(f"Required Education: {jd_info['required_education']}")
        print(f"{'='*80}\n")

        # Process all resumes
        results = []
        pdf_files = [f for f in os.listdir(resume_folder) if f.endswith('.pdf')]
        if not pdf_files:
            print("No PDF files found in the folder!")
            return pd.DataFrame(), jd_info, pd.DataFrame()

        print(f"Processing {len(pdf_files)} resumes...\n")

        # --- Prepare for Cosine Similarity ---
        resume_texts = []
        valid_pdf_files = []
        for pdf_file in pdf_files:
            pdf_path = os.path.join(resume_folder, pdf_file)
            resume_text = self.extract_text_from_pdf(pdf_path)
            if resume_text:
                resume_texts.append(resume_text)
                valid_pdf_files.append(pdf_file)
            else:
                 print(f"Skipping {pdf_file}: Could not extract text.")
        if not resume_texts:
            print("No resumes could be read.")
            return pd.DataFrame(), jd_info, pd.DataFrame()

        jd_processed = jd_info['processed_text']
        resumes_processed = [self.preprocess_text(t) for t in resume_texts]

        corpus = [jd_processed] + resumes_processed
        self.tfidf_vectorizer.fit(corpus)
        jd_vector = self.tfidf_vectorizer.transform([jd_processed])
        resumes_vector = self.tfidf_vectorizer.transform(resumes_processed)

        cosine_scores = cosine_similarity(jd_vector, resumes_vector)[0]

        # --- Loop and analyze each resume ---
        for idx, pdf_file in enumerate(valid_pdf_files):
            print(f"Analyzing {idx+1}/{len(valid_pdf_files)}: {pdf_file}")

            resume_text = resume_texts[idx]
            resume_processed = resumes_processed[idx]
            doc = self.nlp(resume_text)

            # Analyze resume
            resume_info = {
                'text': resume_text,
                'processed_text': resume_processed,
                'contact_info': self.extract_contact_info_and_name(resume_text, doc),
                'skills': self.extract_skills(resume_text),
                'experience': self.extract_experience(resume_text),
                'education': self.extract_education(resume_text, doc)
            }

            cosine_score = cosine_scores[idx]
            # Pass the weights from main() into the scoring function
            scores = self.calculate_composite_score(jd_info, resume_info, cosine_score, weights)

            matched_skills = list(set([s.lower() for s in jd_info['skills']['all_skills']]).intersection(
                set([s.lower() for s in resume_info['skills']['all_skills']])
            ))

            results.append({
                'Resume': pdf_file,
                'Candidate_Name': resume_info['contact_info']['name'],
                'Composite_Score': scores['composite_score'],
                'Cosine_Score': scores['cosine_score'],
                'Skills_Score': scores['skills_score'],
                'Experience_Score': scores['experience_score'],
                'Education_Score': scores['education_score'],
                'Total_Years_Experience': resume_info['experience']['total_years'],
                'Highest_Degree': resume_info['education']['highest_degree'],
                'Matched_Skills': len(matched_skills),
                'Total_JD_Skills': len(jd_info['skills']['all_skills']),
                'Total_Resume_Skills': len(resume_info['skills']['all_skills']),
                'Skills_List': ', '.join(matched_skills),
                'Match_Percentage': (len(matched_skills) / len(jd_info['skills']['all_skills']) * 100) if len(jd_info['skills']['all_skills']) > 0 else 0,
                'Email': resume_info['contact_info']['email'],
                'Phone': resume_info['contact_info']['phone'],
                'Skills_By_Category': json.dumps(resume_info['skills']['by_category'])
            })

        df_results = pd.DataFrame(results)
        df_results = df_results.sort_values('Composite_Score', ascending=False).reset_index(drop=True)

        return df_results.head(top_n), jd_info, df_results # Return all results

    def visualize_results(self, all_results_df, jd_info, output_folder='visualizations'):
        """Create comprehensive visualizations of screening results"""
        if all_results_df.empty:
            print("No results to visualize!")
            return

        results_df = all_results_df.head(15) # Use top 15 for readable charts
        os.makedirs(output_folder, exist_ok=True)
        sns.set_style("whitegrid")
        plt.rcParams['figure.figsize'] = (12, 8)

        # 1. Composite Score Comparison (Top N)
        fig, ax = plt.subplots(figsize=(14, max(8, len(results_df) * 0.5)))
        candidates = [f"{row['Candidate_Name']}\n({row['Resume']})" for _, row in results_df.iterrows()]
        scores = results_df['Composite_Score'].values
        colors = sns.color_palette("RdYlGn", len(candidates))
        bars = ax.barh(candidates, scores, color=colors)
        ax.set_xlabel('Composite Score', fontsize=12, fontweight='bold')
        ax.set_title(f'Top {len(results_df)} Candidates - Composite Score Ranking', fontsize=14, fontweight='bold', pad=20)
        ax.set_xlim(0, 1)
        ax.invert_yaxis()
        for i, (bar, score) in enumerate(zip(bars, scores)):
            ax.text(score + 0.01, bar.get_y() + bar.get_height()/2, f'{score:.3f}', va='center', fontsize=10, fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_folder}/1_composite_scores.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/1_composite_scores.png")
        plt.close()

        # 2. Multi-dimensional Score Breakdown (Top N)
        fig, ax = plt.subplots(figsize=(14, 8))
        df_melted = results_df.melt(
            id_vars='Candidate_Name',
            value_vars=['Skills_Score', 'Cosine_Score', 'Experience_Score', 'Education_Score'],
            var_name='Score_Type',
            value_name='Score'
        )
        sns.barplot(x='Candidate_Name', y='Score', hue='Score_Type', data=df_melted, ax=ax, palette='Set2')
        ax.set_xlabel('Candidates', fontsize=12, fontweight='bold')
        ax.set_ylabel('Score', fontsize=12, fontweight='bold')
        ax.set_title('Score Breakdown by Dimension', fontsize=14, fontweight='bold', pad=20)
        ax.tick_params(axis='x', rotation=45)
        plt.setp(ax.get_xticklabels(), ha='right')
        ax.legend(fontsize=10)
        ax.set_ylim(0, 1)
        plt.tight_layout()
        plt.savefig(f'{output_folder}/2_score_breakdown.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/2_score_breakdown.png")
        plt.close()

        # 3. Skills Match Visualization (Top N)
        fig, ax = plt.subplots(figsize=(14, max(8, len(results_df) * 0.5)))
        candidates_short = [row['Candidate_Name'] for _, row in results_df.iterrows()]
        matched = results_df['Matched_Skills'].values
        total_jd = results_df['Total_JD_Skills'].values[0] if len(results_df) > 0 else 0
        unmatched = [total_jd - m for m in matched]
        bars1 = ax.barh(candidates_short, matched, label='Matched Skills', color='#27ae60')
        bars2 = ax.barh(candidates_short, unmatched, left=matched, label='Unmatched Skills (JD)', color='#ecf0f1')
        ax.set_xlabel('Number of Skills', fontsize=12, fontweight='bold')
        ax.set_title(f'Skills Match Analysis (Total Required: {total_jd})', fontsize=14, fontweight='bold', pad=20)
        ax.legend(fontsize=10)
        ax.invert_yaxis()
        for i, (m, u) in enumerate(zip(matched, unmatched)):
            percentage = (m / total_jd * 100) if total_jd > 0 else 0
            ax.text(m/2, i, f'{m}', ha='center', va='center', fontweight='bold', color='white')
            ax.text(m + u + 0.5, i, f'{percentage:.1f}%', ha='left', va='center', fontweight='bold')
        plt.tight_layout()
        plt.savefig(f'{output_folder}/3_skills_match.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/3_skills_match.png")
        plt.close()

        # 4. Experience vs Score Scatter (All Candidates)
        fig, ax = plt.subplots(figsize=(12, 8))
        scatter = ax.scatter(all_results_df['Total_Years_Experience'],
                             all_results_df['Composite_Score'],
                             s=100, alpha=0.6, c=all_results_df['Composite_Score'],
                             cmap='viridis', edgecolors='black', linewidth=1)
        for idx, row in all_results_df.head(5).iterrows():
            ax.annotate(row['Candidate_Name'],
                       (row['Total_Years_Experience'], row['Composite_Score']),
                       xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')
        if jd_info['required_experience'] > 0:
            ax.axvline(x=jd_info['required_experience'], color='red', linestyle='--', label=f'JD Req: {jd_info["required_experience"]} yrs')
            ax.legend()
        ax.set_xlabel('Years of Experience', fontsize=12, fontweight='bold')
        ax.set_ylabel('Composite Score', fontsize=12, fontweight='bold')
        ax.set_title('Experience vs Composite Score (All Candidates)', fontsize=14, fontweight='bold', pad=20)
        plt.colorbar(scatter, ax=ax, label='Composite Score')
        plt.tight_layout()
        plt.savefig(f'{output_folder}/4_experience_vs_score.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/4_experience_vs_score.png")
        plt.close()

        # 5. Education Distribution (All Candidates)
        fig, ax = plt.subplots(figsize=(10, 8))
        edu_counts = all_results_df['Highest_Degree'].value_counts()
        colors_pie = sns.color_palette("Set2", len(edu_counts))
        wedges, texts, autotexts = ax.pie(edu_counts.values, labels=edu_counts.index,
                                          autopct='%1.1f%%', startangle=90,
                                          colors=colors_pie, textprops={'fontsize': 11, 'fontweight': 'bold'})
        ax.set_title('Education Level Distribution (All Candidates)', fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(f'{output_folder}/5_education_distribution.png', dpi=300, bbox_inches='tight')
        print(f"✓ Saved: {output_folder}/5_education_distribution.png")
        plt.close()

        # 6. Radar Chart for Top 3 Candidates
        if len(results_df) >= 3:
            fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
            categories = ['Skills (Jaccard)', 'Semantic (Cosine)', 'Experience', 'Education']
            num_vars = len(categories)
            angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
            angles += angles[:1]
            ax.set_theta_offset(np.pi / 2)
            ax.set_theta_direction(-1)
            ax.set_xticks(angles[:-1])
            ax.set_xticklabels(categories, fontsize=12, fontweight='bold')
            colors_radar = ['#3498db', '#e74c3c', '#2ecc71']
            for idx in range(min(3, len(results_df))):
                row = results_df.iloc[idx]
                values = [row['Skills_Score'], row['Cosine_Score'], row['Experience_Score'], row['Education_Score']]
                values += values[:1]
                ax.plot(angles, values, 'o-', linewidth=2, label=row['Candidate_Name'], color=colors_radar[idx])
                ax.fill(angles, values, alpha=0.15, color=colors_radar[idx])
            ax.set_ylim(0, 1)
            ax.set_title('Top 3 Candidates - Profile Radar', fontsize=14, fontweight='bold', pad=30)
            ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
            plt.tight_layout()
            plt.savefig(f'{output_folder}/6_radar_chart.png', dpi=300, bbox_inches='tight')
            print(f"✓ Saved: {output_folder}/6_radar_chart.png")
            plt.close()

        print(f"\n✓ All visualizations saved in '{output_folder}' folder!")

    def display_results(self, results_df, jd_info):
        """Display detailed screening results"""
        if results_df.empty:
            print("No results to display!")
            return

        print("\n" + "="*100)
        print(f"{'TOP ' + str(len(results_df)) + ' SHORTLISTED CANDIDATES':^100}")
        print("="*100)
        for idx, row in results_df.iterrows():
            rank = idx + 1
            print(f"\n{'RANK #' + str(rank):^100}")
            print("-" * 100)
            print(f"{'Candidate:':<25} {row['Candidate_Name']}")
            print(f"{'Resume File:':<25} {row['Resume']}")
            print(f"{'Email:':<25} {row['Email']}")
            print(f"{'Phone:':<25} {row['Phone']}")
            print()
            print(f"{'SCORES:':<25}")
            print(f"{'  Hybrid Score:':<25} {row['Composite_Score']:.4f} ({row['Composite_Score']*100:.1f}%)")
            print(f"{'  Skills (Jaccard) Score:':<25} {row['Skills_Score']:.4f} ({row['Skills_Score']*100:.1f}%)")
            print(f"{'  Semantic (Cosine) Score:':<25} {row['Cosine_Score']:.4f} ({row['Cosine_Score']*100:.1f}%)")
            print()
            print(f"{'OTHER METRICS (for info):':<25}")
            print(f"{'  Experience Score:':<25} {row['Experience_Score']:.4f} ({row['Experience_Score']*100:.1f}%)")
            print(f"{'  Education Score:':<25} {row['Education_Score']:.4f} ({row['Education_Score']*100:.1f}%)")
            print()
            print(f"{'QUALIFICATIONS:':<25}")
            print(f"{'  Experience:':<25} {row['Total_Years_Experience']} years (JD: {jd_info['required_experience']} yrs)")
            print(f"{'  Education:':<25} {row['Highest_Degree']} (JD: {jd_info['required_education']})")
            print(f"{'  Matched Skills:':<25} {row['Matched_Skills']} of {row['Total_JD_Skills']} ({row['Match_Percentage']:.1f}%)")
            print()
            print(f"{'MATCHED SKILLS:':<25}")
            skills_list = row['Skills_List'].split(', ') if row['Skills_List'] else ["None"]
            for i in range(0, len(skills_list), 5):
                print(f"{'  ':<25} {', '.join(skills_list[i:i+5])}")
        print("\n" + "="*100 + "\n")

    def save_results(self, results_df, output_file='screening_results.csv'):
        """Save detailed results to CSV"""
        results_df.to_csv(output_file, index=False)
        print(f"✓ Results saved to {output_file}")

    def generate_report(self, results_df, jd_info, all_results_df, output_file='screening_report.html'):
        """Generate HTML report with all results"""
        html_content = f"""
        <!DOCTYPE html>
        <html><head><title>Resume Screening Report</title>
            <style>
                body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 40px; background-color: #f5f5f5; }}
                .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; border-radius: 10px; margin-bottom: 30px; box-shadow: 0 4px 15px rgba(0,0,0,0.1); }}
                .header h1 {{ margin: 0; font-size: 32px; }}
                .jd-section {{ background: white; padding: 25px; border-radius: 10px; margin-bottom: 30px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }}
                .candidate {{ background: white; padding: 25px; margin-bottom: 20px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); border-left: 5px solid #667eea; }}
                .rank {{ display: inline-block; background: #667eea; color: white; padding: 8px 15px; border-radius: 20px; font-weight: bold; font-size: 18px; }}
                .score-container {{ display: grid; grid-template-columns: repeat(auto-fit, minmax(120px, 1fr)); gap: 15px; margin: 20px 0; }}
                .score-box {{ background: #f8f9fa; padding: 15px; border-radius: 8px; text-align: center; border-bottom: 3px solid #667eea; }}
                .score-value {{ font-size: 28px; font-weight: bold; color: #667eea; }}
                .score-label {{ color: #666; font-size: 14px; margin-top: 5px; }}
                .skills {{ display: flex; flex-wrap: wrap; gap: 8px; margin-top: 15px; }}
                .skill-tag {{ background: #e3f2fd; color: #1976d2; padding: 5px 12px; border-radius: 15px; font-size: 13px; }}
                .info-row {{ display: flex; justify-content: space-between; padding: 10px 0; border-bottom: 1px solid #eee; }}
                .info-label {{ font-weight: bold; color: #555; }}
            </style>
        </head><body>
            <div class="header">
                <h1>📋 Resume Screening Report</h1>
                <p>Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
            </div>
            <div class="jd-section">
                <h2>📄 Job Description Analysis</h2>
                <div class="info-row"><span class="info-label">Total Required Skills:</span><span>{len(jd_info['skills']['all_skills'])}</span></div>
                <div class="info-row"><span class="info-label">Required Experience:</span><span>{jd_info['required_experience']} years</span></div>
                <div class="info-row"><span class="info-label">Required Education:</span><span>{jd_info['required_education']}</span></div>
                <div class="info-row"><span class="info-label">Total Candidates Screened:</span><span>{len(all_results_df)}</span></div>
            </div>
        """
        for idx, row in results_df.iterrows():
            rank = idx + 1
            skills_list = row['Skills_List'].split(', ') if row['Skills_List'] else []
            skills_html = ''.join([f'<span class="skill-tag">{skill}</span>' for skill in skills_list])
            if not skills_html: skills_html = "<span>No matched skills found.</span>"
            html_content += f"""
            <div class="candidate">
                <span class="rank">Rank #{rank}</span>
                <h2 style="margin-top: 15px;">{row['Candidate_Name']}</h2>
                <p style="color: #666;">{row['Resume']}</p>
                <div class="score-container">
                    <div class="score-box"><div class="score-value">{row['Composite_Score']:.3f}</div><div class="score-label">Hybrid Score</div></div>
                    <div class="score-box"><div class="score-value">{row['Skills_Score']:.3f}</div><div class="score-label">Skills (Jaccard)</div></div>
                    <div class="score-box"><div class="score-value">{row['Cosine_Score']:.3f}</div><div class="score-label">Semantic (Cosine)</div></div>
                    <div class="score-box"><div class="score-value">{row['Experience_Score']:.3f}</div><div class="score-label">Experience</div></div>
                    <div class="score-box"><div class="score-value">{row['Education_Score']:.3f}</div><div class="score-label">Education</div></div>
                </div>
                <div class="info-row"><span class="info-label">📧 Email:</span><span>{row['Email']}</span></div>
                <div class="info-row"><span class="info-label">📱 Phone:</span><span>{row['Phone']}</span></div>
                <div class="info-row"><span class="info-label">💼 Experience:</span><span>{row['Total_Years_Experience']} years</span></div>
                <div class="info-row"><span class="info-label">🎓 Education:</span><span>{row['Highest_Degree']}</span></div>
                <div class="info-row"><span class="info-label">✅ Skills Match:</span><span>{row['Matched_Skills']} of {row['Total_JD_Skills']} ({row['Match_Percentage']:.1f}%)</span></div>
                <h3 style="margin-top: 20px;">Matched Skills:</h3>
                <div class="skills">{skills_html}</div>
            </div>
            """
        html_content += """</body></html>"""
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"✓ HTML report saved to {output_file}")

    # ---
    # --- NEW FUNCTION TO REPLICATE YOUR PAPER'S EVALUATION ---
    # ---
    def evaluate_extraction_accuracy(self, resume_folder, ground_truth_file):
        """
        Evaluates skill extraction accuracy against a manual ground truth file.
        This is a "meta-analysis" function and is not part of the core screening.

        :param resume_folder: Path to the folder with PDF resumes.
        :param ground_truth_file: Path to a JSON file with the format:
                                  { "resume_filename.pdf": ["skill1", "skill2", ...], ... }
        """
        print("\n" + "="*80)
        print("PERFORMING SKILL EXTRACTION EVALUATION (from Section 4.3.2)")
        print("="*80)

        try:
            with open(ground_truth_file, 'r') as f:
                ground_truth = json.load(f)
            print(f"Loaded ground truth for {len(ground_truth)} resumes from '{ground_truth_file}'")
        except Exception as e:
            print(f"Error: Could not load ground truth file '{ground_truth_file}'. {e}")
            print("Evaluation aborted. Please create this file to run the evaluation.")
            return

        total_correctly_extracted = 0
        total_ground_truth_skills = 0

        for filename, true_skills in ground_truth.items():
            pdf_path = os.path.join(resume_folder, filename)
            if not os.path.exists(pdf_path):
                print(f"Warning: {filename} from ground truth not found in resume folder. Skipping.")
                continue

            # Extract text and skills using the script's functions
            text = self.extract_text_from_pdf(pdf_path)
            if not text:
                print(f"Warning: Could not read {filename}. Skipping.")
                continue

            extracted_skills_data = self.extract_skills(text)
            extracted_skills = set([s.lower() for s in extracted_skills_data['all_skills']])

            # Compare to ground truth
            true_skills_set = set([s.lower() for s in true_skills])
            correctly_extracted = extracted_skills.intersection(true_skills_set)

            total_correctly_extracted += len(correctly_extracted)
            total_ground_truth_skills += len(true_skills_set)

            # Optional: Per-file accuracy
            # accuracy = (len(correctly_extracted) / len(true_skills_set)) * 100 if len(true_skills_set) > 0 else 0
            # print(f"  -> Evaluating: {filename} - Found {len(correctly_extracted)} of {len(true_skills_set)} skills. ({accuracy:.1f}%)")

        if total_ground_truth_skills == 0:
            print("No ground truth skills were found to evaluate against.")
            return

        overall_accuracy = (total_correctly_extracted / total_ground_truth_skills) * 100
        print("\n" + "-"*80)
        print("Overall Skill Extraction Accuracy Results:")
        print(f"  Total Skills in Ground Truth (all files): {total_ground_truth_skills}")
        print(f"  Total Skills Correctly Extracted:         {total_correctly_extracted}")
        print(f"  OVERALL ACCURACY:                         {overall_accuracy:.2f}%")
        print("-" * 80)

        return overall_accuracy

def main():
    """Main function to run the advanced resume screening system"""
    print("="*100)
    print(f"{'ADVANCED RESUME SCREENING SYSTEM':^100}")
    print(f"{'with TF-IDF, NER, Experience Extraction & Visualization':^100}")
    print("="*100)
    print()

    # Initialize screener
    # --- Make sure to use the correct path for your Colab environment ---
    SKILLS_DB_PATH = '/content/drive/MyDrive/CDS_PROJECT/skills_db.json'
    screener = AdvancedResumeScreener(skills_db_path=SKILLS_DB_PATH)

    # Example job description (from your prompt)
    default_jd = """
    Data Analyst

    We are looking for a Data Analyst.
    - Experience with SQL and databases like PostgreSQL or MySQL.
    - Strong skills in data visualization tools like Tableau and Power BI.
    - Proficient in Python, particularly with libraries like Pandas and Numpy.
    - Knowledge of data analysis and statistical models.
    - Good communication and teamwork skills.
    - Bachelor's degree in a related field.
    """

    # Get inputs
    print("=" * 100)
    print("STEP 1: JOB DESCRIPTION")
    print("=" * 100)
    print("\nOptions:")
    print("1. Use default job description (Data Analyst)")
    print("2. Enter custom job description")
    print("3. Load from file")

    choice = input("\nEnter your choice (1/2/3) [default: 1]: ").strip()
    if choice == '2':
        print("\nEnter job description (press Ctrl+D on Linux/macOS or Ctrl+Z+Enter on Windows when done):")
        job_description = ""
        try:
            while True:
                line = input()
                job_description += line + "\n"
        except EOFError:
            pass
    elif choice == '3':
        jd_file = input("Enter path to job description file: ").strip()
        try:
            with open(jd_file, 'r', encoding='utf-8') as f:
                job_description = f.read()
        except:
            print("Error reading file. Using default JD.")
            job_description = default_jd
    else:
        job_description = default_jd

    # Get resume folder
    print("\n" + "=" * 100)
    print("STEP 2: RESUME FOLDER")
    print("=" * 100)
    resume_folder = input("\nEnter path to resume folder [default: ./resumes]: ").strip()
    if not resume_folder:
        resume_folder = "/content/drive/MyDrive/CDS_PROJECT/Resumes" # Defaulting to your Colab path
    if not os.path.exists(resume_folder):
        print(f"\nFolder '{resume_folder}' not found! Please check the path.")
        return

    # Get top N
    print("\n" + "=" * 100)
    print("STEP 3: SCREENING PARAMETERS")
    print("=" * 100)
    top_n_input = input("\nEnter number of top candidates to shortlist [default: 5]: ").strip()
    top_n = int(top_n_input) if top_n_input.isdigit() and int(top_n_input) > 0 else 5

    # --- MODIFIED WEIGHTS SECTION ---
    # Now defaults to the formula from your paper
    print("\n" + "=" * 100)
    print("STEP 4: SCORING WEIGHTS")
    print("=" * 100)
    print("\nUse custom scoring weights?")
    print("Default is based on your paper (0.6 * Cosine + 0.4 * Jaccard/Skills).")
    custom_weights = input("(y/n) [default: n]: ").strip().lower()

    weights = None
    if custom_weights == 'y':
        print("\nEnter weights (must sum to 1.0):")
        try:
            skills_w = float(input("  Skills (Jaccard) weight [default: 0.4]: ") or 0.4)
            cosine_w = float(input("  Semantic (Cosine) weight [default: 0.6]: ") or 0.6)
            exp_w = float(input("  Experience weight [default: 0.0]: ") or 0.0)
            edu_w = float(input("  Education weight [default: 0.0]: ") or 0.0)

            total_w = skills_w + cosine_w + exp_w + edu_w
            if abs(total_w - 1.0) < 0.01:
                weights = {'skills': skills_w, 'cosine': cosine_w, 'experience': exp_w, 'education': edu_w}
                print(f"Using custom weights. Total: {total_w}")
            else:
                print(f"Weights sum to {total_w}, not 1.0. Using defaults from paper.")
                weights = {'skills': 0.4, 'cosine': 0.6}
        except:
            print("Invalid input. Using default weights from paper.")
            weights = {'skills': 0.4, 'cosine': 0.6}
    else:
         print("✓ Using default weights from paper (0.6 * Cosine + 0.4 * Jaccard).")
         weights = {'skills': 0.4, 'cosine': 0.6}

    # --- END OF MODIFIED SECTION ---

    # Process resumes
    print("\n" + "=" * 100)
    print("PROCESSING RESUMES...")
    print("=" * 100)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        top_results, jd_info, all_results = screener.screen_resumes(job_description, resume_folder, top_n, weights)

    if top_results.empty:
        print("\n❌ No resumes could be processed!")
        return

    # Display results
    screener.display_results(top_results, jd_info)

    # Save results
    print("\n" + "=" * 100)
    print("SAVING RESULTS...")
    print("=" * 100)
    screener.save_results(all_results, 'screening_results.csv') # Save all results

    # Generate HTML report
    screener.generate_report(top_results, jd_info, all_results, 'screening_report.html')

    # Generate visualizations
    print("\n" + "=" * 100)
    print("GENERATING VISUALIZATIONS...")
    print("=" * 100)
    visualize = input("\nGenerate visualizations? (y/n) [default: y]: ").strip().lower()
    if visualize != 'n':
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            screener.visualize_results(all_results, jd_info, 'visualizations')

    # ---
    # --- NEW (OPTIONAL) STEP: RUN EVALUATION ---
    # ---
    print("\n" + "=" * 100)
    print("STEP 5: EVALUATE ACCURACY (from Section 4.3.2)")
    print("=" * 100)
    run_eval = input("\nRun skill extraction accuracy evaluation? (y/n) [default: n]: ").strip().lower()
    if run_eval == 'y':
        ground_truth_path = input("  Enter path to ground_truth.json file: ").strip()
        if not ground_truth_path:
             ground_truth_path = "/content/drive/MyDrive/CDS_PROJECT/ground_truth.json" # Example

        screener.evaluate_extraction_accuracy(resume_folder, ground_truth_path)
    # --- END OF NEW STEP ---

    print("\n" + "="*100)
    print(f"{'✓ SCREENING COMPLETED SUCCESSFULLY!':^100}")
    print("="*100)
    print("\nGenerated Files:")
    print("  📊 screening_results.csv - Detailed results for ALL candidates")
    print("  📄 screening_report.html - Interactive report of TOP candidates")
    if visualize != 'n':
        print("  📈 visualizations/ - Folder containing all charts and graphs")
    print("\n" + "="*100)


if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                  ADVANCED RESUME SCREENING SYSTEM                                  
                      with TF-IDF, NER, Experience Extraction & Visualization                       

Loading skills database from /content/drive/MyDrive/CDS_PROJECT/skills_db.json...
STEP 1: JOB DESCRIPTION

Options:
1. Use default job description (Data Analyst)
2. Enter custom job description
3. Load from file

Enter your choice (1/2/3) [default: 1]: 1

STEP 2: RESUME FOLDER

Enter path to resume folder [default: ./resumes]: /content/drive/MyDrive/CDS_PROJECT/Resumes

STEP 3: SCREENING PARAMETERS

Enter number of top candidates to shortlist [default: 5]: 2

STEP 4: SCORING WEIGHTS

Use custom scoring weights?
Default is based on your paper (0.6 * Cosine + 0.4 * Jaccard/Skills).
(y/n) [default: n]: N
✓ Using default weights from paper (0.6 * Cosine + 0.4 * Jaccard).

PROCESSING RESUMES...
Parsing Job Description...

JOB DESCRIPTION ANALYSIS
Required Skills: 12
Skills: power bi, teamwor

# **/content/drive/MyDrive/CDS_PROJECT/ground_truth_example.json**

{
  "Stuti-Resume.pdf": [
    "Python",
    "Machine Learning",
    "TensorFlow",
    "PyTorch",
    "SQL",
    "Git",
    "Docker",
    "AWS",
    "GCP",
    "Agile",
    "Scrum",
    "Communication",
    "CI/CD",
    "scikit-learn"
  ],
  "Om_Patel_CV.pdf": [
    "Python",
    "C++",
    "Java",
    "React",
    "Node.js",
    "MySQL",
    "MongoDB",
    "Git",
    "Communication"
  ],
  "Het_Resume_NEW.pdf": [
    "Python",
    "Pandas",
    "Numpy",
    "SQL",
    "Tableau",
    "Power BI",
    "Excel",
    "Data Analysis"
  ]
}